# Dependências

In [ ]:
# Install required Google Cloud packages (commented out as these are typically one-time setup commands)
# !pip install gcloud
# !gcloud auth application-default login

# Import necessary Python libraries
import pandas as pd                # Data manipulation and analysis
import numpy as np                 # Numerical computing
import time                        # Time-related functions
import os                          # Operating system interfaces
import pandas_gbq                  # Pandas integration with BigQuery
from google.cloud import bigquery  # BigQuery client library
import glob                        # File path pattern matching
import openpyxl                    # Excel file handling
import csv                         # CSV file handling
import re                          # Regular expressions

# Note: The actual imports remain exactly as in the original code

# Tratamento

In [ ]:
# Load Public Safety data from Excel file
df = pd.read_excel("Base_Estadic_2023(2).xlsx", sheet_name='Segurança Pública')

# Select relevant columns and preview first 10 rows
df = df[['Sigla UF', 'Cod UF', 'ESEG01', 'ESEG04', 'ESEG05', 'ESEG06', 'ESEG07']]
df['ano'] = 2023  # Add year column

# Rename columns for clarity
df = df.rename(columns={
    'Sigla UF': 'sigla_uf',
    'Cod UF': 'cod_uf',
    'ESEG01': 'caracterizacao_orgao_gestor',
    'ESEG04': 'genero',
    'ESEG05': 'idade',
    'ESEG06': 'cor_raca',
    'ESEG07': 'grau_instrucao'
})

# Standardize text capitalization
df['caracterizacao_orgao_gestor'] = df['caracterizacao_orgao_gestor'].str.title()

# Define function to correct common typos in organization names
def transformar(nome):
    nome = re.sub(r"\bSeg. \b", "Segurança ", nome)
    nome = re.sub(r"\bSocail\b", "Social", nome)
    nome = re.sub(r"\Segursança\b", "Segurança", nome)
    nome = re.sub(r"\Daa\b", "Da", nome)
    return nome

# Apply text corrections to organization names
df['caracterizacao_orgao_gestor'] = df['caracterizacao_orgao_gestor'].apply(transformar)

# Handle missing/refused data - standardize to 'Sem dados'
for col in ['caracterizacao_orgao_gestor', 'genero', 'cor_raca', 'grau_instrucao']:
    df[col] = np.where(df[col].isin(['Recusa', 'Não Informou', 'Não informou']),
                     'Sem dados',
                     df[col])

# Convert age to numeric and handle missing values
df['idade'] = np.where(df['idade'].isin(['Recusa', 'Não informou']), np.nan, df['idade'])
df['idade'] = pd.to_numeric(df['idade'])

# Standardize race/color category
df['cor_raca'] = np.where(df['cor_raca'] == 'Pardo', 'Parda', df['cor_raca'])

# Create age groups
limites = [18, 30, 50, 65, 100]
categorias = ['Entre 18-29', 'Entre 30-49', 'Entre 50-64', 'Acima de 65']
df['faixa_etaria'] = pd.cut(df['idade'], bins=limites, labels=categorias, right=False)

# Select final columns
df = df[['ano', 'sigla_uf', 'cod_uf', 'caracterizacao_orgao_gestor',
         'genero', 'faixa_etaria', 'cor_raca', 'grau_instrucao']]

# Standardize education levels
dict_esco = {
    'Ensino superior completo': 'Até Ensino Superior Completo',
    'Especialização': 'Até Pós Graduação ou Mestrado',
    'Mestrado': 'Até Pós Graduação ou Mestrado',
    'Doutorado': 'Até Doutorado'
}
df = df.replace({'grau_instrucao': dict_esco})

# Preview final data and structure
df.head()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27 entries, 0 to 26
Data columns (total 8 columns):
 #   Column                       Non-Null Count  Dtype   
---  ------                       --------------  -----   
 0   ano                          27 non-null     int64   
 1   sigla_uf                     27 non-null     object  
 2   cod_uf                       27 non-null     int64   
 3   caracterizacao_orgao_gestor  27 non-null     object  
 4   genero                       27 non-null     object  
 5   faixa_etaria                 25 non-null     category
 6   cor_raca                     27 non-null     object  
 7   grau_instrucao               27 non-null     object  
dtypes: category(1), int64(2), object(5)
memory usage: 1.8+ KB


# Upload

In [ ]:
# Define the table schema for BigQuery with field types and descriptions
schema=[
    bigquery.SchemaField('ano','INTEGER',description='Ano da apuração daquele dado'),
    bigquery.SchemaField('sigla_uf','STRING',description='sigla da UF'),
    bigquery.SchemaField('cod_uf','INTEGER',description='Código do IBGE da UF'),
    bigquery.SchemaField('caracterizacao_orgao_gestor','STRING',description='Caracterização do órgão no qual o gestor está'),
    bigquery.SchemaField('genero','STRING',description='Gênero autodeclarado ou não'),
    bigquery.SchemaField('faixa_etaria','STRING',description='faixa etária da observação'),
    bigquery.SchemaField('cor_raca','STRING',description='Raça/cor da pessoa observada'),
    bigquery.SchemaField('grau_instrucao','STRING',description='Escolaridade da pessoa ou do vínculo observado com detalhamento na pós-graduação')
]

# Initialize the BigQuery client with the specified project
client = bigquery.Client(project='repositoriodedadosgpsp')

# Create a reference to the target dataset
dataset_ref = client.dataset('cargos_lideranca')

# Create a reference to the target table with standardized naming convention
table_ref = dataset_ref.table('ESTADIC_perfil_gestor_seguranca_publica_tipo_orgao_v1')

# Configure the load job with the defined schema
job_config = bigquery.LoadJobConfig(schema=schema)

# Execute the load job to upload the DataFrame to BigQuery
job = client.load_table_from_dataframe(df, table_ref, job_config=job_config)

# Wait for the job to complete
job.result()